# 부경대학교 학사일정을 크롤링하여 구글 캘린더에 연동하기

In [1]:
"""
아래 코드는 현재 페이지의 학사일정을 스크래핑 합니다.
"""


from selenium import webdriver
from bs4 import BeautifulSoup
import time

# 웹 드라이버 설정 및 페이지 로드
driver = webdriver.Chrome()
driver.get('https://www.pknu.ac.kr/main/31')
time.sleep(3) # 페이지가 로드될 시간을 기다립니다.

# 페이지 소스 가져오기
html = driver.page_source

# BeautifulSoup으로 페이지 파싱
soup = BeautifulSoup(html, 'html.parser')

# 학사일정 정보를 담고 있는 테이블의 tbody를 찾습니다.
schedule_table = soup.find('tbody', id='scheduleList')

# 테이블의 모든 행(tr)을 찾습니다.
rows = schedule_table.find_all('tr')

# 날짜와 일정을 저장할 리스트
schedule_data = []

# 각 행에서 날짜와 일정을 추출합니다.
for row in rows:
    date = row.find('td').get_text(strip=True)
    event = row.find_all('td')[1].get_text(strip=True)
    schedule_data.append((date, event))

# 결과 출력
for date, event in schedule_data:
    print(f"날짜: {date}, 일정: {event}")

# 드라이버 종료
driver.quit()

날짜: 2024-5-1 (수), 일정: 근로자의 날
날짜: 2024-5-7 (화) ~ 2024-5-9 (목), 일정: 봄축제
날짜: 2024-5-10 (금), 일정: 개교기념일
날짜: 2024-5-20 (월) ~ 2024-5-22 (수), 일정: 하계 계절수업 수강신청
날짜: 2024-5-22 (수), 일정: 제1학기 수업일수 2/3선


In [2]:
"""
아래 코드는 학사일정 캘린더를 다음 달로 넘겨가면서
더이상 등록된 학사일정이 없을때까지
전체 학사일정을 스크래핑 합니다.
"""


from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

def scrape_schedule(soup):
    schedule_table = soup.find('tbody', id='scheduleList')
    rows = schedule_table.find_all('tr')
    schedule_data = []

    for row in rows:
        date = row.find('td').get_text(strip=True)
        event = row.find_all('td')[1].get_text(strip=True)
        schedule_data.append((date, event))

    return schedule_data

def scrape_next_month_schedule(driver):
    try:
        next_button = driver.find_element(By.XPATH, '//*[@id="calendar"]/tbody[1]/tr[1]/td[3]/a')
        next_button.click()
        time.sleep(3) # 페이지가 로드될 시간을 기다립니다.

        # 페이지 소스 가져오기
        html = driver.page_source

        # BeautifulSoup으로 페이지 파싱
        soup = BeautifulSoup(html, 'html.parser')

        # "등록된 일정이 없습니다." 확인
        no_schedule = soup.find('td', colspan="2", class_="text-center")
        if no_schedule and no_schedule.get_text(strip=True) == "등록된 일정이 없습니다.":
            print("더 이상 일정이 없습니다.")
            return False

        # 일정 스크래핑 및 결과 출력
        schedule_data = scrape_schedule(soup)
        for date, event in schedule_data:
            print(f"날짜: {date}, 일정: {event}")

        return True

    except Exception as e:
        print(f"다음 달 버튼을 클릭하던 중 오류 발생: {e}")
        return False

# 웹 드라이버 설정 및 페이지 로드
driver = webdriver.Chrome()
driver.get('https://www.pknu.ac.kr/main/31')
time.sleep(3) # 페이지가 로드될 시간을 기다립니다.

# 현재 페이지의 일정 스크래핑
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
schedule_data = scrape_schedule(soup)

# 결과 출력
for date, event in schedule_data:
    print(f"날짜: {date}, 일정: {event}")

# 이후 달의 일정 스크래핑
while scrape_next_month_schedule(driver):
    pass

# 드라이버 종료
driver.quit()

날짜: 2024-5-1 (수), 일정: 근로자의 날
날짜: 2024-5-7 (화) ~ 2024-5-9 (목), 일정: 봄축제
날짜: 2024-5-10 (금), 일정: 개교기념일
날짜: 2024-5-20 (월) ~ 2024-5-22 (수), 일정: 하계 계절수업 수강신청
날짜: 2024-5-22 (수), 일정: 제1학기 수업일수 2/3선
날짜: 2024-4-29 (월) ~ 2024-6-12 (수), 일정: 제1학기 대학원 학위청구 논문발표
날짜: 2024-6-10 (월) ~ 2024-6-20 (목), 일정: 제1학기 보강지정
날짜: 2024-6-10 (월) ~ 2024-7-1 (월), 일정: 제1학기 강의평가
날짜: 2024-6-14 (금), 일정: 제1학기 대학원 학위청구 논문 심사완료
날짜: 2024-6-21 (금) ~ 2024-6-27 (목), 일정: 제1학기 기말고사
날짜: 2024-6-21 (금) ~ 2024-7-1 (월), 일정: 제1학기 성적입력
날짜: 2024-6-21 (금) ~ 2024-7-4 (목), 일정: 제1학기 성적공시
날짜: 2024-6-28 (금), 일정: 하계방학
날짜: 2024-6-10 (월) ~ 2024-7-1 (월), 일정: 제1학기 강의평가
날짜: 2024-6-21 (금) ~ 2024-7-1 (월), 일정: 제1학기 성적입력
날짜: 2024-6-21 (금) ~ 2024-7-4 (목), 일정: 제1학기 성적공시
날짜: 2024-7-1 (월) ~ 2024-7-19 (금), 일정: 하계 계절수업
날짜: 2024-7-2 (화) ~ 2024-7-4 (목), 일정: 제1학기 성적이의신청 및 정정
날짜: 2024-7-5 (금), 일정: (담당교원) 제1학기 성적확정
날짜: 2024-7-8 (월), 일정: 제1학기 최종 성적확정
날짜: 2024-7-10 (수) ~ 2024-7-12 (금), 일정: 제2학기 다전공 이수(취소) 및 전과 신청
날짜: 2024-7-15 (월) ~ 2024-7-19 (금), 일정: 제1학기 대학원 학위청구 논문 완

In [3]:
"""
아래 코드는 전체 학사 일정을 스크래핑하고
그 결과를 리스트에 담습니다.
"""


from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

def scrape_schedule(soup):
    schedule_table = soup.find('tbody', id='scheduleList')
    rows = schedule_table.find_all('tr')
    schedule_data = []

    for row in rows:
        date = row.find('td').get_text(strip=True)
        event = row.find_all('td')[1].get_text(strip=True)
        schedule_data.append((date, event))

    return schedule_data

def scrape_next_month_schedule(driver):
    try:
        next_button = driver.find_element(By.XPATH, '//*[@id="calendar"]/tbody[1]/tr[1]/td[3]/a')
        next_button.click()
        time.sleep(3) # 페이지가 로드될 시간을 기다립니다.

        # 페이지 소스 가져오기
        html = driver.page_source

        # BeautifulSoup으로 페이지 파싱
        soup = BeautifulSoup(html, 'html.parser')

        # "등록된 일정이 없습니다." 확인
        no_schedule = soup.find('td', colspan="2", class_="text-center")
        if no_schedule and no_schedule.get_text(strip=True) == "등록된 일정이 없습니다.":
            return None

        # 일정 스크래핑
        schedule_data = scrape_schedule(soup)
        
        return schedule_data

    except Exception as e:
        print(f"다음 달 버튼을 클릭하던 중 오류 발생: {e}")
        return None

# 웹 드라이버 설정 및 페이지 로드
driver = webdriver.Chrome()
driver.get('https://www.pknu.ac.kr/main/31')
time.sleep(3) # 페이지가 로드될 시간을 기다립니다.

# 현재 페이지의 일정 스크래핑
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
event_data_list = scrape_schedule(soup)

# 이후 달의 일정 스크래핑 및 리스트에 추가
while True:
    schedule_data = scrape_next_month_schedule(driver)
    if schedule_data is None:
        break
    event_data_list.extend(schedule_data)

# 드라이버 종료
driver.quit()

# 스크래핑한 일정 리스트 출력
print(event_data_list)

[('2024-5-1 (수)', '근로자의 날'), ('2024-5-7 (화) ~ 2024-5-9 (목)', '봄축제'), ('2024-5-10 (금)', '개교기념일'), ('2024-5-20 (월) ~ 2024-5-22 (수)', '하계 계절수업 수강신청'), ('2024-5-22 (수)', '제1학기 수업일수 2/3선'), ('2024-4-29 (월) ~ 2024-6-12 (수)', '제1학기 대학원 학위청구 논문발표'), ('2024-6-10 (월) ~ 2024-6-20 (목)', '제1학기 보강지정'), ('2024-6-10 (월) ~ 2024-7-1 (월)', '제1학기 강의평가'), ('2024-6-14 (금)', '제1학기 대학원 학위청구 논문 심사완료'), ('2024-6-21 (금) ~ 2024-6-27 (목)', '제1학기 기말고사'), ('2024-6-21 (금) ~ 2024-7-1 (월)', '제1학기 성적입력'), ('2024-6-21 (금) ~ 2024-7-4 (목)', '제1학기 성적공시'), ('2024-6-28 (금)', '하계방학'), ('2024-6-10 (월) ~ 2024-7-1 (월)', '제1학기 강의평가'), ('2024-6-21 (금) ~ 2024-7-1 (월)', '제1학기 성적입력'), ('2024-6-21 (금) ~ 2024-7-4 (목)', '제1학기 성적공시'), ('2024-7-1 (월) ~ 2024-7-19 (금)', '하계 계절수업'), ('2024-7-2 (화) ~ 2024-7-4 (목)', '제1학기 성적이의신청 및 정정'), ('2024-7-5 (금)', '(담당교원) 제1학기 성적확정'), ('2024-7-8 (월)', '제1학기 최종 성적확정'), ('2024-7-10 (수) ~ 2024-7-12 (금)', '제2학기 다전공 이수(취소) 및 전과 신청'), ('2024-7-15 (월) ~ 2024-7-19 (금)', '제1학기 대학원 학위청구 논문 완성본 제출'), ('2024-7-17 (수)

In [4]:
pip install icalendar

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [5]:
"""
아래 코드는 스크래핑한 학사일정을 이용해
ical 파일을 생성합니다.
"""


import icalendar
from icalendar import Event, vDatetime
from datetime import datetime, timedelta
import pytz

def generate_ical_event(event_data):
    event = Event()

    # 이벤트 제목 설정
    event.add('summary', event_data[1])

    # 날짜 및 시간 파싱
    date_range = event_data[0].split(' ~ ')
    start_date = datetime.strptime(date_range[0].split(' ')[0], '%Y-%m-%d')
    end_date = datetime.strptime(date_range[-1].split(' ')[0], '%Y-%m-%d')

    # 시간대 설정
    timezone = pytz.timezone('Asia/Seoul')
    start_date = timezone.localize(start_date)
    end_date = timezone.localize(end_date)

    event.add('dtstart', start_date)
    event.add('dtend', end_date)

    return event

def generate_ical_file(event_data_list, file_name='calendar.ics'):
    cal = icalendar.Calendar()

    for event_data in event_data_list:
        event = generate_ical_event(event_data)
        cal.add_component(event)

    with open(file_name, 'wb') as f:
        f.write(cal.to_ical())

# 예시: 학사일정 데이터를 icalendar 형식으로 변환하여 ical 파일 생성
generate_ical_file(event_data_list, 'Academic_calendar.ics')
print("ical 파일이 생성되었습니다.")

ical 파일이 생성되었습니다.
